In [1]:
# import dependencies
import pandas as pd
import datetime as dt

In [2]:
# set path by which to find the source data
filepath = '../Resources/indexData.csv'

In [3]:
# read in the file via pandas
index_data = pd.read_csv(filepath)

In [4]:
# change the value in the Date column to datetime format
for index, value in enumerate(index_data['Date']):
    index_data.loc[index, 'Date'] = dt.datetime.fromisoformat(value)

In [5]:
# print to check to see it looks right
index_data

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31 00:00:00,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03 00:00:00,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04 00:00:00,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05 00:00:00,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06 00:00:00,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27 00:00:00,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28 00:00:00,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31 00:00:00,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01 00:00:00,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [6]:
# drop nans
index_data = index_data.dropna(how='any')

In [7]:
# find the rows within the 2018-2021 time range
index_data = index_data.loc[index_data['Date'] >= dt.datetime.strptime('2018-01-01','%Y-%m-%d')
]

In [8]:
# find the indices in the file
index_data['Index'].unique()

array(['NYA', 'IXIC', 'HSI', '000001.SS', 'GSPTSE', '399001.SZ', 'NSEI',
       'GDAXI', 'KS11', 'SSMI', 'TWII', 'J203.JO', 'N225', 'N100'],
      dtype=object)

In [9]:
# write helper function to do the groupby and find aggregate answers

def group_agg(index_name):
    
    '''
    takes in an index name and creates a dataframe that contains statistics grouped by year and month for that index
    '''
    
    index_data_filt = index_data.loc[index_data['Index'] == str(index_name)]

    avgopen = index_data_filt.groupby(pd.Grouper(key="Date", freq="1M")).mean()['Open']

    avgclose = index_data_filt.groupby(pd.Grouper(key = "Date",freq = "1M")).mean()['Close']

    maxh = index_data_filt.groupby(pd.Grouper(key = "Date", freq = "1M")).max()['High']

    minh = index_data_filt.groupby(pd.Grouper(key = "Date", freq = "1M")).max()['Low']

    #find the total maximum mkt capitalization (of constituents of the index) in a given month
    #by finding the highest value product of the volume multiplied by the highest price

    maxvol = index_data_filt.groupby(pd.Grouper(key = "Date", freq = "1M")).max()['Volume']

    max_mktcap = maxh*maxvol
    
    return pd.DataFrame(
        {
        "High": maxh,
        "Low": minh,
        "Open": avgopen,
        "Close": avgclose,
        "Marketcap": max_mktcap
        }
    )

In [10]:
# apply the function to all the indices

NYA_agg = group_agg('NYA')
IXIC_agg = group_agg('IXIC')
oneSS_agg = group_agg('000001.SS')
GSPTSE_agg = group_agg('GSPTSE')
SZ_agg = group_agg('399001.SZ')
GDAXI_agg = group_agg('GDAXI')
KS11_agg = group_agg('KS11')
SSMI_agg = group_agg('SSMI')
TWII_agg = group_agg('TWII')
J203_agg = group_agg('J203.JO')
N225_agg = group_agg('N225')
N100_agg = group_agg('N100')
HSI_agg = group_agg('HSI')
NSEI_agg = group_agg('NSEI')

In [11]:
# add in the index

NYA_agg['Symbol'] = 'NYA'
IXIC_agg['Symbol'] = 'IXIC'

oneSS_agg['Symbol'] = '000001.SS'
GSPTSE_agg['Symbol'] = 'GSPTSE'

SZ_agg['Symbol'] = '399001.SZ'
GDAXI_agg['Symbol'] = 'GDAXI'

KS11_agg['Symbol'] = 'KS11'
SSMI_agg['Symbol'] = 'SSMI'

TWII_agg['Symbol'] = 'TWII'
J203_agg['Symbol'] = 'J203.JO'

N225_agg['Symbol'] = 'N225'
N100_agg['Symbol'] = 'N100'

HSI_agg['Symbol'] = 'HSI'
NSEI_agg['Symbol'] = 'NSEI'


In [12]:
# reset index

NYA_agg = NYA_agg.reset_index()
IXIC_agg = IXIC_agg.reset_index()

oneSS_agg = oneSS_agg.reset_index()
GSPTSE_agg = GSPTSE_agg.reset_index()

SZ_agg = SZ_agg.reset_index()
GDAXI_agg = GDAXI_agg.reset_index()

KS11_agg = KS11_agg.reset_index()
SSMI_agg = SSMI_agg.reset_index()

TWII_agg = TWII_agg.reset_index()
J203_agg = J203_agg.reset_index()

N225_agg = N225_agg.reset_index()
N100_agg = N100_agg.reset_index()

HSI_agg = HSI_agg.reset_index()
NSEI_agg = NSEI_agg.reset_index()


In [13]:
# helper function to transform the datetime column values to integers for merging in the future

def to_int(df):
    '''
    takes in a dataframe and changes the Date column values from datetime objects to integers showing year and month
    '''
    for index, x in enumerate(df['Date']):
        x = x.strftime('%Y%m')
        df.loc[index, 'Date'] = int(x)
    
    return df


In [14]:
# apply the to_int helper function to the indices to change the datetime value to integer format
NYA_agg = to_int(NYA_agg)
IXIC_agg = to_int(IXIC_agg)
oneSS_agg = to_int(oneSS_agg)
GSPTSE_agg = to_int(GSPTSE_agg)
SZ_agg = to_int(SZ_agg)
GDAXI_agg = to_int(GDAXI_agg)
KS11_agg = to_int(KS11_agg)
SSMI_agg = to_int(SSMI_agg)
TWII_agg = to_int(TWII_agg)
J203_agg = to_int(J203_agg)
N225_agg = to_int(N225_agg)
N100_agg = to_int(N100_agg)
HSI_agg = to_int(HSI_agg)
NSEI_agg = to_int(NSEI_agg)

In [15]:
# add in the index and name of the index

NYA_agg['Name'] = 'NYSE Composite Index'

IXIC_agg['Name'] = 'NASDAQ Composite Index'

oneSS_agg['Name'] = 'Shanghai Stock Exchange Composite Index'

GSPTSE_agg['Name'] = 'S&P/TSX Composite Index'

SZ_agg['Name'] = 'Shenzhen Composite Index'

GDAXI_agg['Name'] = 'DAX Composite Index'

KS11_agg['Name'] = 'Korean Composite Stock Price Index'

SSMI_agg['Name'] = 'Swiss Market Index'

TWII_agg['Name'] = 'Taiwan Weighted Index'

J203_agg['Name'] = 'Johannesburg Stock Exchange Index'

N225_agg['Name'] = 'Nikkei Index'

N100_agg['Name'] = 'Euronext 100 Index' 

HSI_agg['Name'] = 'Hang Seng Index'

NSEI_agg['Name'] = 'National Stock Exchange of India Index'


In [16]:
# round all the integer values to 3 decimal places

NYA_agg = NYA_agg.round(3)
IXIC_agg = IXIC_agg.round(3)

oneSS_agg = oneSS_agg.round(3)
GSPTSE_agg = GSPTSE_agg.round(3)

SZ_agg = SZ_agg.round(3)
GDAXI_agg = GDAXI_agg.round(3)

KS11_agg = KS11_agg.round(3)
SSMI_agg = SSMI_agg.round(3)

TWII_agg = TWII_agg.round(3)
J203_agg = J203_agg.round(3)

N225_agg = N225_agg.round(3)
N100_agg = N100_agg.round(3)

HSI_agg = HSI_agg.round(3)
NSEI_agg = NSEI_agg.round(3)


In [17]:
# write the individual tables to csv files

NYA_agg.to_csv('cleaned csv outputs/NYA_agg.csv', index=False)
IXIC_agg.to_csv('cleaned csv outputs/IXIC_agg.csv', index=False)

oneSS_agg.to_csv('cleaned csv outputs/oneSS_agg.csv', index=False)
GSPTSE_agg.to_csv('cleaned csv outputs/GSPTSE_agg.csv', index=False)

SZ_agg.to_csv('cleaned csv outputs/SZ_agg.csv', index=False)
GDAXI_agg.to_csv('cleaned csv outputs/GDAXI_agg.csv', index=False)

KS11_agg.to_csv('cleaned csv outputs/KS11_agg.csv', index=False)
SSMI_agg.to_csv('cleaned csv outputs/SSMI_agg.csv', index=False)

TWII_agg.to_csv('cleaned csv outputs/TWII_agg.csv', index=False)
J203_agg.to_csv('cleaned csv outputs/J203_agg.csv', index=False)

N225_agg.to_csv('cleaned csv outputs/N225_agg.csv', index=False)
N100_agg.to_csv('cleaned csv outputs/N100_agg.csv', index=False)

HSI_agg.to_csv('cleaned csv outputs/HSI_agg.csv', index=False)
NSEI_agg.to_csv('cleaned csv outputs/NSEI_agg.csv', index=False)